In [8]:
import pandas as pd
blog=pd.read_csv('blogtext.csv')


In [9]:
blog=blog[:9000]

In [10]:
from tqdm import tqdm
for i in tqdm(range(len(blog))):
    if len(blog['text'][i])<100:
        blog.drop([i], axis=0, inplace=True)

100%|██████████| 9000/9000 [00:01<00:00, 5055.21it/s]


In [11]:
len(blog)

7481

In [12]:
blog=blog[:6592]

In [17]:
blog.index = list(range(0,6592))

In [18]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('all-MiniLM-L6-v2')
embedding=model.encode(blog['text'])

In [19]:
import torch
emb = torch.tensor(embedding)

In [20]:
torch.save(emb,"embedding.pt")

In [21]:
e=torch.load('embedding.pt')

In [22]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
n_gram_range = (3, 3)
top_n = 5
style_embedding=[]
model=SentenceTransformer('all-MiniLM-L6-v2')
for i in tqdm(range(len(blog))):
    count = CountVectorizer(ngram_range=n_gram_range,stop_words=None).fit([blog['text'][i]])
    candidates = count.get_feature_names_out()
    candidate_embeddings = model.encode(candidates)
    distances=cosine_similarity([embedding[i]], candidate_embeddings)
    keywords_embeddings = np.array([candidate_embeddings[index] for index in distances.argsort()[0][-top_n:]])
    mean_keywords=np.mean(keywords_embeddings,axis=0)
    style_embedding.append(embedding[i]-mean_keywords)

from sklearn.cluster import KMeans
cluster=10
kmeans=KMeans(n_clusters=cluster)
style=np.array(style_embedding)
kmeans.fit(style)
label=kmeans.labels_

100%|██████████| 6592/6592 [38:10<00:00,  2.88it/s]  
C:\Users\dofod\anaconda3\envs\NLP\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [23]:
from sklearn.metrics import v_measure_score
v_measure_score(blog["id"],label)

0.1581790230523081

In [25]:
lab = torch.tensor(label)
torch.save(lab,"label.pt")